In [1]:
import numpy as np
import tensorflow as tf

Building Fish Farm

In [2]:
T=3.0
N=int(T*2*12)
Nsim = 10
dtype=np.float32
seed=1

t=np.linspace(0,T,N*Nsim,endpoint=True,dtype=dtype).reshape((-1,1))
r=0.0303

In [3]:
from Commodities import Schwartz2Factor
'Salmon'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.01, 0.9, 0.57, 95] # down,down
salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.2, 0.9, 0.57, 95] # down,up
# salmonParam=[0.12, 0.23, 0.75, 2.6, 0.02, 0.6, 0.9, 0.57, 95] # up,up

'Soy'
# mu, sigma1, sigma2, kappa, alpha, lambda, rho, delta0, P0
# soyParam=[0.15, 0.5, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # low vol
soyParam=[0.15, 1, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # medium vol
# soyParam=[0.15, 2, 0.4, 1.2, 0.06, 0.14, 0.44, 0.0, 1] # high vol

'Risk neutral dynamics'
salmonParam[0]=r
soyParam[0]=r

"Fish feeding 25% of production cost, disease 30%, harvest 10%. Total production cost = 50% of price = labor, smolt, ..."
salmonPrice=salmonParam[-1] #NOK/KG
harvestingCosts=salmonPrice*0.5*0.1 # roughly 10%
feedingCosts=salmonPrice*0.5*0.25
initialSalmon=0.5*salmonPrice+feedingCosts+harvestingCosts #we add the costs to salmon price since they are respected in the model, other costs are fixed and thus removed
salmonParam[-1]=initialSalmon
print(f'Feeding costs {feedingCosts} and Harvesting costs {harvestingCosts}')
soyParam[-1]=feedingCosts # to save the right dataset, since initial price is not relevant for soy model

rngSoy=np.random.default_rng(seed*100+1)
soy=Schwartz2Factor(soyParam,t,dtype=dtype,rng=rngSoy)

rngSalmon=np.random.default_rng(seed*100+2)
salmon=Schwartz2Factor(salmonParam,t,dtype=dtype,rng=rngSalmon)

Feeding costs 11.875 and Harvesting costs 4.75


In [4]:
from Harvest import Harvest
hc = harvestingCosts
harvest = Harvest(hc)

In [5]:
from Growth import Bertalanffy
wInf=6
a=1.113
b=1.097
c=1.43
growth = Bertalanffy(t,wInf,a,b,c)

In [6]:
from Price import Price
price = Price(salmon)

In [7]:
from Feed import StochFeed,DetermFeed
cr=1.1
fc=feedingCosts
# feed = StochFeed(fc,cr,r,t,soy)
feed = DetermFeed(fc,cr,r,t,soy)

In [8]:
from Mortality import ConstMortatlity
n0=10000
m=0.1
mort = ConstMortatlity(t,n0,m)

In [9]:
from FishFarm import fishFarm
farm = fishFarm(growth,feed,price,harvest,mort,stride=Nsim)

In [10]:
from OptimalStopping import Polynomial,LSMC
deg=2
batch_size=2**13 #2**13=8192
batches=32 #maybe not process safe, because same rng for all processes #TODO compare to regular loop
gen = farm.generateFishFarm

basis = Polynomial(deg=deg,dtype=dtype)

opt=LSMC(r,farm.tCoarse,gen,basis)
# opt.train(batches)

rngSoy=np.random.default_rng(seed*100+1)
rngSalmon=np.random.default_rng(seed*100+2)
salmon.setgen(rngSalmon)
soy.setgen(rngSoy)
tau,Vtau=opt.evaluate(batch_size,batches)

print(f'Mean stopping time {np.mean(tau)} with mean value {np.mean(Vtau)}')

Mean stopping time 2.49592661857605 with mean value 2414047.75
